### Naive/persistence model
Assumes each day's day prices predict the next in the same hour

In [60]:
import json
import csv
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon
from shapely.affinity import scale
import matplotlib.pyplot as plt
import glob
import os
import datetime
import pytz
from pytz import timezone

In [61]:
#Loading AS price data; downselecting to reg down prices
prices = pd.read_csv("../data/AS_price_vol.csv")
price = prices[['date','hr_beg','price_DAH_REGDN']]

In [63]:
#Lagging by 24 hours
price_24 = price['price_DAH_REGDN'].shift(24)
df = pd.concat([price,price_24], axis=1)
df.columns = ['date','hr_beg','price','price_24']

In [67]:
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

In [69]:
#Downselecting to 2018
df['year'] = pd.to_datetime(df['date']).dt.year
df_2018 = df[df['year']==2018]

In [72]:
#Calculating number of spikes captured in 2018
spike = np.mean(df['price'][df['year']<2018]) + np.std(df['price'][df['year']<2018])
df_2018['Actual spike'] = df_2018['price'] >= spike
df_2018['Predicted spike'] = df_2018['price_24'] >= spike
df_2018['Both spike'] = (df_2018['price'] >= spike) & (df_2018['price_24'] >= spike)
df_2018['Neither spike'] = (df_2018['price'] < spike) & (df_2018['price_24'] < spike)

In [81]:
spikes_correct = sum(df_2018['Both spike']) / sum(df_2018['Actual spike'])
overall_correct = (sum(df_2018['Both spike']) + sum(df_2018['Neither spike'])) / len(df_2018)
print("Spikes correctly predicted, 2018: " + str(spikes_correct))
print("Overall hours correctly predicted, 2018: " + str(overall_correct))

Spikes correctly predicted, 2018: 0.5407407407407407
Overall hours correctly predicted, 2018: 0.9718229523157654


In [82]:
#Calculating number of spikes captured, 2008-2017
df['Actual spike'] = df['price'] >= spike
df['Predicted spike'] = df['price_24'] >= spike
df['Both spike'] = (df['price'] >= spike) & (df['price_24'] >= spike)
df['Neither spike'] = (df['price'] < spike) & (df['price_24'] < spike)

In [83]:
spikes_correct = sum(df['Both spike']) / sum(df['Actual spike'])
overall_correct = (sum(df['Both spike']) + sum(df['Neither spike'])) / len(df)
print("Spikes correctly predicted, 2018: " + str(spikes_correct))
print("Overall hours correctly predicted, 2018: " + str(overall_correct))

Spikes correctly predicted, 2018: 0.6701503221188261
Overall hours correctly predicted, 2018: 0.95222108240853
